# XGBoost forecasting pipeline for PKD sectors

This notebook builds an XGBoost-based forecasting pipeline that:
1. Loads and maps PKD sectors to their corresponding commodity features.
2. Prepares aligned feature/target datasets per PKD.
3. Trains and evaluates models on historical data (2012–2022 train, 2023–2024 test).
4. Uses the trained models to forecast PKD variables for years 2025–2027.
5. Saves evaluation metrics and forecasts to CSV files.


In [1]:

import json
import os

import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [2]:

# Paths (adjust if you move the notebook or data files)
PROJECT_DIR = r"/Users/jan/Documents/competitions/hackatons/Hacknation2025"
DATA_DIR = r"/Users/jan/Documents/competitions/hackatons/Hacknation2025/data"
PKD_TO_VARS_PATH = os.path.join(PROJECT_DIR, "pkd_to_variables.json")
COMMODITIES_PATH = os.path.join(DATA_DIR, "COMMODITIES_FINAL.xlsx")
TARGETS_PATH = os.path.join(DATA_DIR, "df_pivot_filtered_numbers.csv")

# Time split configuration (assumption based on problem description)
YEAR_COL = "year"
TRAIN_START = 2012
TRAIN_END = 2022   # inclusive
TEST_START = 2023  # inclusive
TEST_END = 2024    # inclusive
FORECAST_YEARS = [2025, 2026, 2027]

# Output paths
EVAL_OUTPUT_PATH = os.path.join(DATA_DIR, "xgb_evaluation_results.csv")
FORECAST_OUTPUT_PATH = os.path.join(DATA_DIR, "xgb_forecasts_2025_2027.csv")

# -------------------------------------------------------------------------
# Load mapping from PKD codes to commodity variables
# -------------------------------------------------------------------------
with open(PKD_TO_VARS_PATH, "r", encoding="utf-8") as f:
    pkd_to_vars = json.load(f)

# -------------------------------------------------------------------------
# Load commodities and target data
# -------------------------------------------------------------------------
df_com = pd.read_excel(COMMODITIES_PATH)
df_target = pd.read_csv(TARGETS_PATH)

print("Commodities shape:", df_com.shape)
print("Targets shape:", df_target.shape)
print("Commodities years:", df_com[YEAR_COL].min(), "->", df_com[YEAR_COL].max())
print("Targets years:", df_target[YEAR_COL].min(), "->", df_target[YEAR_COL].max())


Commodities shape: (16, 32)
Targets shape: (13, 571)
Commodities years: 2012 -> 2027
Targets years: 2012 -> 2024


In [3]:

# Ensure year is integer and sort
df_com[YEAR_COL] = df_com[YEAR_COL].astype(int)
df_target[YEAR_COL] = df_target[YEAR_COL].astype(int)

df_com = df_com.sort_values(YEAR_COL).reset_index(drop=True)
df_target = df_target.sort_values(YEAR_COL).reset_index(drop=True)

# Try to coerce non-year columns in commodities to numeric where possible
for col in df_com.columns:
    if col == YEAR_COL:
        continue
    if df_com[col].dtype == object:
        # Example: 'It zarobki' has non-breaking spaces and comma decimal separator
        df_com[col] = (
            df_com[col]
            .astype(str)
            .str.replace("\xa0", "", regex=False)
            .str.replace(" ", "", regex=False)
            .str.replace(",", ".", regex=False)
        )
        df_com[col] = pd.to_numeric(df_com[col], errors="coerce")

# Quick sanity check
print("dtypes (commodities):")
print(df_com.dtypes.head(15))


dtypes (commodities):
year                                                                                     int64
przewieziona masa towarów mln ton                                                      float64
Interest rates (yr close)                                                              float64
Function of costs of budowa                                                            float64
Pozwolenia wydane na budowe                                                            float64
It zarobki                                                                             float64
Polietylen                                                                             float64
Sprzedaż hurtowa - dynamika                                                            float64
Aluminium                                                                              float64
Średnia cena za metr w WWA                                                             float64
Średnia roczna cena sprzedaż

In [4]:

def get_feature_columns_for_pkd(pkd_code, df_com, pkd_to_vars):
    """Return (available_features, missing_features) for a given PKD code."""
    requested = pkd_to_vars.get(pkd_code, [])
    available = [c for c in requested if c in df_com.columns]
    missing = sorted(set(requested) - set(available))
    return available, missing


def get_target_columns_for_pkd(pkd_code, df_target):
    """
    Get all target columns belonging to a PKD.
    Assumption: columns are of the form '<PKD>_<something>', e.g. '10.1_CF'.
    """
    prefix = f"{pkd_code}_"
    target_cols = [c for c in df_target.columns if c.startswith(prefix)]
    return target_cols


def prepare_pkd_dataset(pkd_code, df_com, df_target, pkd_to_vars):
    """
    Build merged feature/target DataFrame for a given PKD, aligned on year.

    Returns
    -------
    df_merged : pd.DataFrame
        Contains YEAR_COL, target columns, and feature columns.
    feature_cols : list of str
    target_cols : list of str
    missing_features : list of str
    """
    feature_cols, missing_features = get_feature_columns_for_pkd(pkd_code, df_com, pkd_to_vars)
    target_cols = get_target_columns_for_pkd(pkd_code, df_target)

    if not feature_cols:
        print(f"[WARN] PKD {pkd_code}: no available commodity features. Skipping.")
        return None, [], [], missing_features

    if not target_cols:
        print(f"[WARN] PKD {pkd_code}: no target columns found. Skipping.")
        return None, feature_cols, [], missing_features

    df_t = df_target[[YEAR_COL] + target_cols]
    df_c = df_com[[YEAR_COL] + feature_cols]

    df_merged = pd.merge(df_t, df_c, on=YEAR_COL, how="inner").sort_values(YEAR_COL)
    return df_merged, feature_cols, target_cols, missing_features


def time_based_split(df_merged, feature_cols, target_cols):
    """
    Split the merged PKD dataset into train/test based on calendar years.
    """
    mask_train = (df_merged[YEAR_COL] >= TRAIN_START) & (df_merged[YEAR_COL] <= TRAIN_END)
    mask_test = (df_merged[YEAR_COL] >= TEST_START) & (df_merged[YEAR_COL] <= TEST_END)

    X_train = df_merged.loc[mask_train, feature_cols].copy()
    y_train = df_merged.loc[mask_train, target_cols].copy()

    X_test = df_merged.loc[mask_test, feature_cols].copy()
    y_test = df_merged.loc[mask_test, target_cols].copy()

    # Drop rows where any target is missing
    train_valid_mask = y_train.notna().all(axis=1)
    test_valid_mask = y_test.notna().all(axis=1)

    X_train, y_train = X_train[train_valid_mask], y_train[train_valid_mask]
    X_test, y_test = X_test[test_valid_mask], y_test[test_valid_mask]

    return X_train, X_test, y_train, y_test




In [8]:
def make_xgb_model(random_state=42):
    """Create a base XGBRegressor with reasonable default hyperparameters."""
    model = XGBRegressor(
        n_estimators=500,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=random_state,
    )
    return model


def evaluate_predictions(y_true, y_pred, target_cols):
    """
    Compute MAE, RMSE, and R^2 per target column.
    RMSE is computed as sqrt(MSE) to avoid using the `squared` kwarg,
    which may not be available in older sklearn versions.
    """
    results = []
    for idx, col in enumerate(target_cols):
        mae = mean_absolute_error(y_true.iloc[:, idx], y_pred[:, idx])
        
        # Older sklearn versions do not support `squared` kwarg
        mse = mean_squared_error(y_true.iloc[:, idx], y_pred[:, idx])
        rmse = np.sqrt(mse)
        
        r2 = r2_score(y_true.iloc[:, idx], y_pred[:, idx])
        results.append((col, mae, rmse, r2))
    return results

In [12]:

eval_rows = []
models = {}  # optional: store models per PKD
forecast_df = pd.DataFrame({YEAR_COL: FORECAST_YEARS})

for pkd_code in sorted(pkd_to_vars.keys()):
    print(f"\n================ PKD {pkd_code} ================")
    df_merged, feature_cols, target_cols, missing_features = prepare_pkd_dataset(
        pkd_code, df_com, df_target, pkd_to_vars
    )

    if df_merged is None or not feature_cols or not target_cols:
        continue

    print(f"Features ({len(feature_cols)}):", feature_cols)
    if missing_features:
        print(f"[INFO] Missing feature columns for {pkd_code} (ignored): {missing_features}")

    # Time-based train/test split
    X_train, X_test, y_train, y_test = time_based_split(df_merged, feature_cols, target_cols)
    print("Train shape:", X_train.shape, y_train.shape)
    print("Test shape:", X_test.shape, y_test.shape)

    if X_train.empty or y_train.empty:
        print(f"[WARN] PKD {pkd_code}: empty training set, skipping.")
        continue

    base_model = make_xgb_model(random_state=42)
    model = MultiOutputRegressor(base_model)

    # Convert to NumPy arrays so XGBoost does not see pandas column names
    X_train_np = X_train.to_numpy()
    y_train_np = y_train.to_numpy()
    X_test_np = X_test.to_numpy() if not X_test.empty else None

    model.fit(X_train_np, y_train_np)
    models[pkd_code] = model

    # Evaluation on test set (if available)
    if (X_test_np is not None) and (not y_test.empty):
        y_pred_test = model.predict(X_test_np)
        results = evaluate_predictions(y_test, y_pred_test, target_cols)

        for target_name, mae, rmse, r2 in results:
            eval_rows.append(
                {
                    "pkd_code": pkd_code,
                    "target": target_name,
                    "n_train": len(X_train),
                    "n_test": len(X_test),
                    "mae": mae,
                    "rmse": rmse,
                    "r2": r2,
                    "n_features_used": len(feature_cols),
                    "missing_features": ";".join(missing_features),
                }
            )
    else:
        print(f"[INFO] PKD {pkd_code}: no valid test data for evaluation.")

    # ------------------------------------------------------------------
    # Forecast 2025–2027 for this PKD
    # ------------------------------------------------------------------
    df_future = df_com[df_com[YEAR_COL].isin(FORECAST_YEARS)].copy()
    df_future = df_future.sort_values(YEAR_COL)

    df_future = df_com[df_com[YEAR_COL].isin(FORECAST_YEARS)].copy()
    df_future = df_future.sort_values(YEAR_COL)

    if df_future.empty:
        print(f"[WARN] PKD {pkd_code}: no commodity data for forecast years, skipping forecast.")
        continue

    X_future = df_future[feature_cols]
    X_future_np = X_future.to_numpy()
    y_future_pred = model.predict(X_future_np)

    df_pkd_forecast = pd.DataFrame(y_future_pred, columns=target_cols)
    df_pkd_forecast.insert(0, YEAR_COL, df_future[YEAR_COL].values)

    # Merge into the global forecast DataFrame (wide format, like the original targets)
    forecast_df = forecast_df.merge(df_pkd_forecast, on=YEAR_COL, how="left")

# Convert evaluation results to DataFrame
eval_df = pd.DataFrame(eval_rows)
print("\nEvaluation results summary (head):")
display(eval_df.head())

print("\nForecasts (head):")
display(forecast_df.head())



================ PKD 10.1 ================
Features (6): ['żywiec bydło', 'mleko krowie 1l', 'pszenica 1 dt', 'CPI', 'usd/pln', 'eur/pln']
[INFO] Missing feature columns for 10.1 (ignored): ['Export_mln_PLN']
Train shape: (5, 6) (5, 38)
Test shape: (2, 6) (2, 38)

================ PKD 22.2 ================
Features (6): ['Polietylen', 'GAZ ZIEMNY_close', 'Węgiel_USD', 'usd/pln', 'eur/pln', 'CPI']
[INFO] Missing feature columns for 22.2 (ignored): ['Export_mln_PLN']
Train shape: (5, 6) (5, 38)
Test shape: (2, 6) (2, 38)

================ PKD 29.3 ================
Features (6): ['Samochody osobowe ogółem', 'HICP - Motor cars', 'Aluminium', 'usd/pln', 'eur/pln', 'Interest rates (yr close)']
[INFO] Missing feature columns for 29.3 (ignored): ['Export_mln_PLN']
Train shape: (4, 6) (4, 38)
Test shape: (2, 6) (2, 38)

================ PKD 35.1 ================
Features (9): ['Średnia roczna cena sprzedaży energii elektrycznej na rynku konkurencyjnym [zł/MWh]', 'GAZ ZIEMNY_close', 'Węgiel_USD

,pkd_code,target,n_train,n_test,mae,rmse,r2,n_features_used,missing_features
0,10.1,10.1_CF,5,2,1102.791133,1547.858586,-18.727948,6,Export_mln_PLN
1,10.1,10.1_EN,5,2,7.088837,7.905809,-9.000291,6,Export_mln_PLN
2,10.1,10.1_GS,5,2,25344.265625,28993.617108,-4036.242401,6,Export_mln_PLN
3,10.1,10.1_GS_I,5,2,25448.523750,29016.530120,-3855.491105,6,Export_mln_PLN
4,10.1,10.1_LTC,5,2,527.073506,733.635348,-0.586130,6,Export_mln_PLN



Forecasts (head):


,year,10.1_CF,10.1_EN,10.1_GS,10.1_GS_I,10.1_LTC,10.1_LTL,10.1_NP,10.1_PEN,10.1_PNPM,...,62.0_C_to_GS,62.0_INV_to_GS_I,62.0_REC_to_GS_I,62.0_REC_to_STL,62.0_INV_to_STL,62.0_NWC_to_STL,62.0_CF_to_TC,62.0_OFE_to_GS,62.0_OFE_to_NP,62.0_PPO_to_GS_I
0,2025,4865.680664,571.588806,79748.210938,78394.617188,3483.694580,4483.138184,2676.704102,500.496338,76384.867188,...,0.124856,0.014678,0.232035,1.16012,0.076157,0.738189,0.114482,0.004403,0.067054,0.012571
1,2026,4865.680664,571.588806,79748.210938,78394.617188,3483.694580,4483.138184,2676.704102,500.496338,76384.867188,...,0.124856,0.014678,0.232035,1.16012,0.076157,0.738189,0.114482,0.004403,0.067054,0.012571
2,2027,6340.997070,573.588867,106999.257812,105340.695312,3628.118408,4768.445801,3631.274170,498.344849,102171.195312,...,0.124856,0.014678,0.232035,1.16012,0.076157,0.738189,0.114482,0.004403,0.067054,0.012571


In [13]:

# Save evaluation metrics and forecasts to CSV
if not eval_df.empty:
    eval_df.to_csv(EVAL_OUTPUT_PATH, index=False)
    print("Saved evaluation results to:", EVAL_OUTPUT_PATH)
else:
    print("[INFO] No evaluation results to save.")

forecast_df.to_csv(FORECAST_OUTPUT_PATH, index=False)
print("Saved forecasts to:", FORECAST_OUTPUT_PATH)


Saved evaluation results to: /Users/jan/Documents/competitions/hackatons/Hacknation2025/data/xgb_evaluation_results.csv
Saved forecasts to: /Users/jan/Documents/competitions/hackatons/Hacknation2025/data/xgb_forecasts_2025_2027.csv
